In [5]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import openai
import time
from dotenv import load_dotenv
load_dotenv() 

False

## MP3 downloaden

* z.B. über https://notube.cc/de/youtube-app-v74
* ggf. noch ffmpeg/ffprobe installieren (mac: `brew install ffmpeg`)

## In Einzelteile zerlegen

In [3]:
# Chunk up the audio file 
sound_file = AudioSegment.from_mp3('podcast/Podcast.mp3')

In [10]:
chunksize=len(sound_file)/20
start=0
i=0
end=start+chunksize
ges_transcript=[]

while end<len(sound_file):
    print(start,end)
    chunk=sound_file[start:end]
    chunk.export('podcast/Podcast_'+str(i)+'.mp3',format='mp3')
    with open('podcast/Podcast_'+str(i)+'.mp3', 'rb') as podcast_part:
        transcript = openai.Audio.translate("whisper-1", podcast_part, prompt="In this episode, my guest is Tim Ferriss — a five-time #1 New York Times bestselling author, technology investor and host of the iconic podcast, The Tim Ferriss Show. We discuss Tim’s process of exploration, experimentation and mastery — themes that have spanned his career that have placed him on the cutting-edge of many important fields. Tim explains what questions to ask when approaching any new endeavor in order to maximize success. He also explains how to incorporate structure and playfulness into skill and knowledge mastery, how to find and work with mentors, the key importance of location and networks in creating truly impactful things. We also discuss Tims philanthropic efforts to support research on psychedelics for the treatment of mental health challenges and we discuss his latest creative endeavors. This episode should be of interest to a wide range of listeners, as Tim’s mastery and wisdom spans athletic and mental pursuits, business, media, technology and the arts. What distinguishes Tim is his ability to thoughtfully deconstruct these processes in order to teach others how to do the same.")
    ges_transcript.append(transcript)
    print(transcript.text[:200], '..\n', transcript.text[-200:])
    start=end
    end=start+chunksize
    i+=1

0 657417.15
Welcome to the Huberman Lab Podcast, where we discuss science and science-based tools for everyday life. I'm Andrew Huberman and I'm a professor of neurobiology and ophthalmology at Stanford School of ..
  esoteric? Because it is all played out very well. What I'm basically saying is if you want to know what's going to be happening hot and useful in five years, you need to go back and look at the book.
657417.15 1314834.3
Tim Ferriss, thank you for the very generous comparison and intro. I'm thrilled to be here. So thanks for having me. The four-hour body represented an opportunity for me to do a few things. The first  ..
 ng right now? Cobbling together awkward solutions. Was that exploration a structured practice for you, or is this just something that was the consequence of being Tim Ferriss waking up in the morning?
1314834.3 1972251.4500000002
TIM CARUSO we'd like to introduce Tim Ferris for this podcast. of structure in the first place? It's like, okay, now it's time 

## Zusammenfassen

In [31]:
from langchain import PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import HypotheticalDocumentEmbedder

In [17]:
splitter=CharacterTextSplitter(chunk_size = 15000)
docs=splitter.create_documents([transcript.text for transcript in ges_transcript])
len(docs)

19

In [20]:
prompt_template = """Write a concise summary of the following transcript of the Huberman Lab Podcast with Tim Ferriss. The summary should be concise and only include the main takeaways:


{text}


CONCISE SUMMARY IN GERMAN:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point:\n\n {existing_answer}\n\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in German. Use Markdown to format your answer.\n"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(ChatOpenAI(model_name='gpt-3.5-turbo-16k-0613', temperature=0.2), chain_type="refine", return_intermediate_steps=False, question_prompt=PROMPT, refine_prompt=refine_prompt)

In [21]:
summary= chain({"input_documents": docs}, return_only_outputs=True)

In [22]:
print(summary['output_text'])

Tim Ferriss betont die Bedeutung des bewussten Zeitmanagements, der Auswahl der richtigen Aktivitäten und Menschen sowie der Fähigkeit, sich auf das Wesentliche zu konzentrieren, um ein erfülltes und erfolgreiches Leben zu führen. Er spricht auch über die Bedeutung von Deoptimierung und Pausen, um Unbehagen zu erleben und neue kreative Energien freizusetzen. Ferriss betont die Vorteile von Mentoren und die Bedeutung des Lernens von fast jedem. Er spricht auch über die Wichtigkeit der Aufmerksamkeitszuweisung und des Zeitmanagements, um sich auf wichtige Aktivitäten zu konzentrieren und Ablenkungen zu vermeiden. Ferriss erwähnt den Einfluss von sozialen Medien auf die Aufmerksamkeit und betont die Notwendigkeit, bewusst mit ihnen umzugehen. Insgesamt betont er die Bedeutung von Energie, Kreativität und der Verfolgung von Projekten, die uns Freude bereiten. Ferriss teilt auch persönliche Erfahrungen, wie seine eigene Suizidgedanken in der Vergangenheit, um anderen zu helfen und ihnen Mut

# Fragen Stellen

In [34]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator=' ')
texts = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()

### HyDe Embeddings für bessere Übersetzung Deutsch/Englisch bei kurzen Queries

In [44]:
hyde_prompt_template = """Please answer the user's question about the podcast transcript. The user question will be in German but the answer should be in English!

Question: {question}
Answer in English:"""
hyde_prompt = PromptTemplate(input_variables=["question"], template=hyde_prompt_template)
llm_chain = LLMChain(llm=ChatOpenAI(), prompt=hyde_prompt)

hyde_embeddings = HypotheticalDocumentEmbedder(
    llm_chain=llm_chain, base_embeddings=embeddings
)
docsearch = Chroma.from_documents(texts, hyde_embeddings)
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name='gpt-3.5-turbo-16k-0613', temperature=0.0), chain_type="stuff", retriever=docsearch.as_retriever())

In [52]:
qa.run("Was sind Tim Feriss Erfahrungen im Bezug auf geistige Gesundheit?")

'Tim Ferriss hat sich intensiv mit dem Thema geistige Gesundheit auseinandergesetzt und ist ein Befürworter der Verwendung von psychedelischen Substanzen zur Behandlung von psychischen Erkrankungen wie Depressionen, Essstörungen und Sucht. Er hat mehrere Millionen Dollar für die Erforschung von psychedelischer Therapie gespendet und hat damit maßgeblich dazu beigetragen, dass dieses Feld von der Randwissenschaft zur etablierten Forschung mit Unterstützung durch Philanthropie und staatliche Fördermittel geworden ist. Tim Ferriss hat sich auch für verschiedene philanthropische Projekte engagiert und setzt sich aktiv für die Verbesserung der geistigen Gesundheit ein.'